# Sistema de recomendaciones basado en Contenido
## Universidad del Valle de Guatemala <br> Facultad de Ingeniería
#### Departamento de Ciencias de la Computación <br> Deep Learning y Sistemas Inteligentes - Sección 20 <br><br> Jeyner Arango 201106 <br> Cristian Laynez 201281


In [60]:
# Importar las librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.models import Model
from keras.layers import Input, Embedding, Concatenate, Flatten, Dense

In [61]:
data_frame = pd.read_csv("./data/_AllData.csv")
data_frame.head()

,User-ID,ISBN,Book-Rating,Book-Title,Location,Age
0,276725,034545104X,0,Flesh Tones: A Novel,"tyler, texas, usa",-1.0
1,276726,0155061224,5,Rites of Passage,"seattle, washington, usa",-1.0
2,276727,0446520802,0,The Notebook,"h, new south wales, australia",16.0
3,276729,052165615X,3,Help!: Level 1,"rijeka, n/a, croatia",16.0
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,"rijeka, n/a, croatia",16.0


### Ahora vamos a construir un modelo para crear el sistema de recomendación

In [62]:
# Preprocesamiento de datos
label_encoder = LabelEncoder()
data_frame['User-ID'] = label_encoder.fit_transform(data_frame['User-ID'])
data_frame['ISBN'] = label_encoder.fit_transform(data_frame['ISBN'])
data_frame['Location'] = label_encoder.fit_transform(data_frame['Location'])
data_frame['Age'] = data_frame['Age'].fillna(data_frame['Age'].mean())  # Manejar valores nulos en Age
# scaler = MinMaxScaler()
# data_frame[['Year-Of-Publication', 'Age']] = scaler.fit_transform(data_frame[['Year-Of-Publication', 'Age']])


In [63]:
# Dividir datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(data_frame, test_size=0.2, random_state=42)

In [64]:
# Definir el modelo
user_input = Input(shape=(1,))
book_input = Input(shape=(1,))
location_input = Input(shape=(1,))
age_input = Input(shape=(1,))

In [65]:
embedding_size = 50

user_embedding = Embedding(input_dim=data_frame['User-ID'].nunique(), output_dim=embedding_size)(user_input)
book_embedding = Embedding(input_dim=data_frame['ISBN'].nunique(), output_dim=embedding_size)(book_input)
location_embedding = Embedding(input_dim=data_frame['Location'].nunique(), output_dim=embedding_size)(location_input)
age_embedding = Embedding(input_dim=1, output_dim=embedding_size)(age_input)

concatenated = Concatenate()([user_embedding, book_embedding, location_embedding, age_embedding])
flattened = Flatten()(concatenated)
dense1 = Dense(128, activation='relu')(flattened)
output = Dense(1)(dense1)

model = Model(inputs=[user_input, book_input, location_input, age_input], outputs=output)

In [66]:
# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

In [67]:
# Entrenar el modelo
model.fit([train_data['User-ID'], train_data['ISBN'], train_data['Location'], train_data['Age']],
          train_data['Book-Rating'],
          epochs=5,
          batch_size=64,
          validation_data=([test_data['User-ID'], test_data['ISBN'], test_data['Location'], test_data['Age']],
                           test_data['Book-Rating']))

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node model_4/embedding_19/embedding_lookup defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\crist\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\crist\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\crist\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\crist\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "d:\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "d:\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "d:\Programs\Python\Python311\Lib\asyncio\events.py", line 80, in _run

  File "

In [ ]:
predictions = model.predict([test_data['User-ID'], test_data['ISBN'], test_data['Location'], test_data['Age']])
mse = mean_squared_error(test_data['Book-Rating'], predictions)
print(f'Mean Squared Error: {mse}')